In [77]:
#Importing Dependencies

import pandas as pd
import requests
import json
import time
from datetime import datetime
import pickle
import psycopg2
from dotenv import load_dotenv 
import os
import great_expectations as ge
from great_expectations.validator.validator import Validator
from great_expectations.execution_engine.pandas_execution_engine import PandasExecutionEngine
from great_expectations.core.batch import Batch
from great_expectations.core import ExpectationSuite
import tempfile


In [49]:

from great_expectations.core.batch import Batch
from great_expectations.execution_engine import PandasExecutionEngine
from great_expectations.core.expectation_suite import ExpectationSuite

In [5]:
#Extracting the data from the api

app_id = os.getenv('APP_ID'),
APi_key = os.getenv('API_KEY'),
POLL_INTERVAL = 60  # Time in seconds between requests

# Define the endpoint and parameters for scheduled data
station_code = 'WAT'  # Example station code (London Waterloo)
url = f'https://transportapi.com/v3/uk/train/station/{station_code}/live.json'

params = {
    'app_id': app_id,
    'api_key': APi_key,
    #'time_of_day': '19:00',
    #'request_time': '2024-10-30T18:50:00+00:00',
    'darwin': 'false',  
    'train_status': 'passenger',  # Status filter, e.g., passenger trains only
    'live' :'True'
}
response = requests.get(url, params=params)

if response.status_code == 200:
    response= response.json()
else:
    print(f"Failed to fetch data: {response.status_code}")

In [6]:
response

{'date': '2024-11-03',
 'time_of_day': '21:32',
 'request_time': '2024-11-03T21:32:38+00:00',
 'station_name': 'London Waterloo',
 'station_code': 'WAT',
 'departures': {'all': [{'mode': 'train',
    'service': '24620104',
    'train_uid': 'L58133',
    'platform': '9',
    'operator': 'SW',
    'operator_name': 'South Western Railway',
    'aimed_departure_time': '21:35',
    'aimed_arrival_time': None,
    'aimed_pass_time': None,
    'origin_name': 'London Waterloo',
    'destination_name': 'Wareham',
    'source': 'Network Rail',
    'category': 'XX',
    'service_timetable': {'id': 'https://transportapi.com/v3/uk/train/service_timetables/L58133:2024-11-03.json?app_id=e2e4d023&app_key=bf7baff0c295d4fc1ab3e121837d3d68&live=true'},
    'status': 'STARTS HERE',
    'expected_arrival_time': None,
    'expected_departure_time': '21:35',
    'best_arrival_estimate_mins': None,
    'best_departure_estimate_mins': 2},
   {'mode': 'train',
    'service': '24629204',
    'train_uid': 'L60828

In [7]:
with open("response.pkl", "wb") as f:
    pickle.dump(response, f)

In [8]:
train_columns = []

#date = response['date']
request_time = response['request_time']
station_name = response['station_name']

train_info = {
    #'date': date,
    'request_time': request_time,
    'station_name': station_name
}
train_columns.append(train_info)

train_columns

[{'request_time': '2024-11-03T21:32:38+00:00',
  'station_name': 'London Waterloo'}]

In [9]:
train_columns_df = pd.DataFrame(train_columns)
train_columns_df.reset_index(inplace=True)
train_columns_df

,index,request_time,station_name
0,0,2024-11-03T21:32:38+00:00,London Waterloo


In [10]:
train__departure_columns = []

for columns in response['departures']['all']:
    try:
        row ={
            'mode':columns['mode'],
            'train_uid': columns['train_uid'],
            'origin_name': columns['origin_name'],
            'operator_name':columns['operator_name'],
            'platform': columns['platform'],
            'destination_name': columns['destination_name'],
            'aimed_departure_time': columns['aimed_departure_time'],
            'expected_departure_time': columns['expected_departure_time'],
            'best_departure_estimate_mins': columns['best_departure_estimate_mins'],
            'aimed_arrival_time': columns['aimed_arrival_time']
        }
        train__departure_columns.append( row)
    except (KeyError, TypeError) as e:
        print(f"Error processing columns: {e}")     

print(train__departure_columns)

[{'mode': 'train', 'train_uid': 'L58133', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '9', 'destination_name': 'Wareham', 'aimed_departure_time': '21:35', 'expected_departure_time': '21:35', 'best_departure_estimate_mins': 2, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L60828', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '12', 'destination_name': 'Basingstoke', 'aimed_departure_time': '21:37', 'expected_departure_time': '21:37', 'best_departure_estimate_mins': 4, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L58728', 'origin_name': 'London Waterloo', 'operator_name': 'South Western Railway', 'platform': '17', 'destination_name': 'Reading', 'aimed_departure_time': '21:39', 'expected_departure_time': '21:39', 'best_departure_estimate_mins': 6, 'aimed_arrival_time': None}, {'mode': 'train', 'train_uid': 'L59428', 'origin_name': 'London Waterloo', 'operator_name': 'South 

In [11]:
train__departure_columns_df = pd.DataFrame(train__departure_columns)
train__departure_columns_df.reset_index(inplace=True)
train__departure_columns_df

,index,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,train,L58133,London Waterloo,South Western Railway,9,Wareham,21:35,21:35,2,None
1,1,train,L60828,London Waterloo,South Western Railway,12,Basingstoke,21:37,21:37,4,None
2,2,train,L58728,London Waterloo,South Western Railway,17,Reading,21:39,21:39,6,None
3,3,train,L59428,London Waterloo,South Western Railway,4,Guildford,21:40,21:40,7,None
4,4,train,L62269,London Waterloo,South Western Railway,16,Windsor & Eton Riverside,21:44,21:44,11,None
5,5,train,L60039,London Waterloo,South Western Railway,1,Shepperton,21:48,21:48,15,None
6,6,train,L57828,London Waterloo,South Western Railway,11,Portsmouth Harbour,21:50,21:50,17,None
7,7,train,L57734,London Waterloo,South Western Railway,18,Woking,21:50,21:50,17,None
8,8,train,L60409,London Waterloo,South Western Railway,5,Hampton Court,21:57,21:57,24,None
9,9,train,L61572,London Waterloo,South Western Railway,12,Haslemere,22:00,22:00,27,None


In [55]:
#Merging both dataframes

raw_train_schedule_df = pd.merge(train_columns_df, train__departure_columns_df, on='index', how='outer' )
raw_train_schedule_df

,index,request_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-11-03T21:32:38+00:00,London Waterloo,train,L58133,London Waterloo,South Western Railway,9,Wareham,21:35,21:35,2,None
1,1,NaN,NaN,train,L60828,London Waterloo,South Western Railway,12,Basingstoke,21:37,21:37,4,None
2,2,NaN,NaN,train,L58728,London Waterloo,South Western Railway,17,Reading,21:39,21:39,6,None
3,3,NaN,NaN,train,L59428,London Waterloo,South Western Railway,4,Guildford,21:40,21:40,7,None
4,4,NaN,NaN,train,L62269,London Waterloo,South Western Railway,16,Windsor & Eton Riverside,21:44,21:44,11,None
5,5,NaN,NaN,train,L60039,London Waterloo,South Western Railway,1,Shepperton,21:48,21:48,15,None
6,6,NaN,NaN,train,L57828,London Waterloo,South Western Railway,11,Portsmouth Harbour,21:50,21:50,17,None
7,7,NaN,NaN,train,L57734,London Waterloo,South Western Railway,18,Woking,21:50,21:50,17,None
8,8,NaN,NaN,train,L60409,London Waterloo,South Western Railway,5,Hampton Court,21:57,21:57,24,None
9,9,NaN,NaN,train,L61572,London Waterloo,South Western Railway,12,Haslemere,22:00,22:00,27,None


In [64]:
from great_expectations.core.batch import Batch, BatchRequest, RuntimeBatchRequest
from great_expectations.execution_engine import PandasExecutionEngine
from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.validator.validator import Validator

In [62]:
from great_expectations import get_context


In [ ]:
#validating the ingested data

try:
    

    suite = ExpectationSuite("raw_train_schedule_suite")

    #raw_execution_engine = PandasExecutionEngine()


    # Set up an execution engine
   

    # Define your DataFrame (assuming train_schedule_df is your DataFrame)
    raw_schedule_df = raw_train_schedule_df

    raw_execution_engine = PandasExecutionEngine()


    # Use a Batch to wrap the DataFrame
    raw_batch = Batch(data=raw_train_schedule_df)

    # Create a Validator with the Batch and ExpectationSuite
    raw_validator = Validator(
        execution_engine=raw_execution_engine,
        batches=[raw_batch],
        expectation_suite=suite
    )

    # Add expectations directly to the Validator
    raw_validator.expect_column_values_to_not_be_null(column="index")
    raw_validator.expect_column_values_to_not_be_null(column="request_time")
    raw_validator.expect_column_values_to_not_be_null(column="station_name")
    raw_validator.expect_column_values_to_not_be_null(column="mode")
    raw_validator.expect_column_values_to_not_be_null(column="train_uid")
    raw_validator.expect_column_values_to_not_be_null(column="origin_name")
    raw_validator.expect_column_values_to_not_be_null(column="operator_name")
    raw_validator.expect_column_values_to_not_be_null(column="platform")
    raw_validator.expect_column_values_to_not_be_null(column="destination_name")
    raw_validator.expect_column_values_to_not_be_null(column="aimed_departure_time")
    raw_validator.expect_column_values_to_not_be_null(column="expected_departure_time")
    raw_validator.expect_column_values_to_not_be_null(column="best_departure_estimate_mins")
    raw_validator.expect_column_values_to_not_be_null(column="aimed_arrival_time") 

    # Step 6: Validate the DataFrame and print the results
    raw_results = raw_validator.validate()
    print("Validation results:", raw_results)

except Exception as e:
    print(f"Validation failed: {e}")

    

c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 317.38it/s] 
c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 499.04it/s] 
c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add

Validation results: {
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "index"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "request_time"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 43,
        "unexpected_percent": 97.72727272727273,
        "partial_unexpected_list": [
          null,
          null,
          n

In [89]:
train_schedule_df = raw_train_schedule_df

In [90]:
train_schedule_df.rename(columns= {'index':'id'}, inplace = True)

In [91]:
#Renaming request time column

train_schedule_df.rename(columns= {'request_time':'request_date_time'}, inplace = True)

In [92]:
train_schedule_df['request_date_time'] = train_schedule_df['request_date_time'].ffill()


In [97]:
train_schedule_df['station_name'] = train_schedule_df['station_name'].ffill()

In [98]:
#Filling up missing values

train_schedule_df['aimed_arrival_time'].fillna('Unknonwn', inplace=True)
train_schedule_df.reset_index(drop=True, inplace=True)
#train_schedule_df = train_schedule_df.loc[:, ~train_schedule_df.columns.str.contains('^level_')]


In [99]:
train_schedule_df

,id,request_date_time,station_name,mode,train_uid,origin_name,operator_name,platform,destination_name,aimed_departure_time,expected_departure_time,best_departure_estimate_mins,aimed_arrival_time
0,0,2024-11-03T21:32:38+00:00,London Waterloo,train,L58133,London Waterloo,South Western Railway,9,Wareham,21:35,21:35,2,Unknonwn
1,1,2024-11-03T21:32:38+00:00,London Waterloo,train,L60828,London Waterloo,South Western Railway,12,Basingstoke,21:37,21:37,4,Unknonwn
2,2,2024-11-03T21:32:38+00:00,London Waterloo,train,L58728,London Waterloo,South Western Railway,17,Reading,21:39,21:39,6,Unknonwn
3,3,2024-11-03T21:32:38+00:00,London Waterloo,train,L59428,London Waterloo,South Western Railway,4,Guildford,21:40,21:40,7,Unknonwn
4,4,2024-11-03T21:32:38+00:00,London Waterloo,train,L62269,London Waterloo,South Western Railway,16,Windsor & Eton Riverside,21:44,21:44,11,Unknonwn
5,5,2024-11-03T21:32:38+00:00,London Waterloo,train,L60039,London Waterloo,South Western Railway,1,Shepperton,21:48,21:48,15,Unknonwn
6,6,2024-11-03T21:32:38+00:00,London Waterloo,train,L57828,London Waterloo,South Western Railway,11,Portsmouth Harbour,21:50,21:50,17,Unknonwn
7,7,2024-11-03T21:32:38+00:00,London Waterloo,train,L57734,London Waterloo,South Western Railway,18,Woking,21:50,21:50,17,Unknonwn
8,8,2024-11-03T21:32:38+00:00,London Waterloo,train,L60409,London Waterloo,South Western Railway,5,Hampton Court,21:57,21:57,24,Unknonwn
9,9,2024-11-03T21:32:38+00:00,London Waterloo,train,L61572,London Waterloo,South Western Railway,12,Haslemere,22:00,22:00,27,Unknonwn


In [100]:
#Saving file to CSV
train_schedule_df.to_csv('train_schedule.csv')

In [101]:
#validating transformed data

try:

    suite = ExpectationSuite("train_schedule_suite")


    # Set up an execution engine
    execution_engine = PandasExecutionEngine()

    # Use a Batch to wrap the DataFrame
    batch = Batch(data=train_schedule_df)

    # Step 3: Create an Expectation Suite
    #suite_name = "train_schedule_suite"
    #suite = ExpectationSuite(expectation_suite_name=suite_name)

    # Create a Validator with the Batch and ExpectationSuite
    validator = Validator(
        execution_engine=execution_engine,
        batches=[batch],
        expectation_suite=suite
    )

    # Add expectations directly to the Validator
    validator.expect_column_values_to_not_be_null(column="id")
    validator.expect_column_values_to_not_be_null(column="request_date_time")
    validator.expect_column_values_to_not_be_null(column="station_name")
    validator.expect_column_values_to_not_be_null(column="mode")
    validator.expect_column_values_to_not_be_null(column="train_uid")
    validator.expect_column_values_to_not_be_null(column="origin_name")
    validator.expect_column_values_to_not_be_null(column="operator_name")
    validator.expect_column_values_to_not_be_null(column="platform")
    validator.expect_column_values_to_not_be_null(column="destination_name")
    validator.expect_column_values_to_not_be_null(column="aimed_departure_time")
    validator.expect_column_values_to_not_be_null(column="expected_departure_time")
    validator.expect_column_values_to_not_be_null(column="best_departure_estimate_mins")
    validator.expect_column_values_to_not_be_null(column="aimed_arrival_time") 

    # Step 6: Validate the DataFrame and print the results
    transformed_results = validator.validate()
    print("Validation results:", transformed_results)

except Exception as e:
    print(f"Data Validation failed{e}")



# Show all expectations added to the suite
#print("Expectations in suite:", validator.get_expectation_suite().expectations)

c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics:  67%|██████▋   | 4/6 [00:00<00:00, 347.89it/s]

Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 407.96it/s]
c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 472.02it/s] 
c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 241.09it/s]
c:\Users\imarr\AppData\Local\Programs\Python\Python312\Lib\site-packages\great_expectations\expectations\expectation.py:1426: UserWarning: `result_format` 

Validation results: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "id"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "request_date_time"
        },
        "meta": {}
      },
      "result": {
        "element_count": 44,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
    

In [105]:
# Database connection details

local_conn = psycopg2.connect(
                        host= 'localhost', 
                        database = 'velocity_railway', 
                        user = 'postgres', 
                        password = 'Password', 
                        port= '5432'
)

# Connect to Azure and Local PostgreSQL
#azure_conn = psycopg2.connect(**azure_conn_details)
#local_conn = psycopg2.connect(local_conn_details)

cursor = local_conn.cursor()

#Create table

#Execute query to create table
cursor.execute("""CREATE TABLE train_schedule(
    id SERIAL PRIMARY KEY,
    request_date_time  VARCHAR (100),
    station_name VARCHAR (100),
    mode VARCHAR (100),
    train_uid VARCHAR (100),
    origin_name VARCHAR (100),
    operator_name VARCHAR (100),
    platform VARCHAR(100),
    destination_name VARCHAR(100),
    aimed_departure_time VARCHAR (100),
    expected_departure_time VARCHAR(100),
    best_departure_estimate_mins INT,
    aimed_arrival_time VARCHAR (100)

);
""")

#committing the query to database
local_conn.commit()

In [109]:
# Data loading function
def load_data_to_db(local_conn , table_name):
    cursor = local_conn.cursor()
    insert_query = f'''
    INSERT INTO train_schedule (id, request_date_time, station_name, mode, train_uid, origin_name, operator_name, platform, \
                                  destination_name,aimed_departure_time, expected_departure_time,best_departure_estimate_mins,aimed_arrival_timeVARCHAR )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s))
    '''
    rows = [tuple(x) for x in table_name[["id, request_date_time, station_name, mode, train_uid, origin_name, operator_name, platform, \
                                  destination_name,aimed_departure_time, expected_departure_time,best_departure_estimate_mins,aimed_arrival_timeVARCHAR" ]].values]
    
    try:
        cursor.executemany(insert_query, rows)
        local_conn.commit()
        print(f"Data loaded successfully into {table_name}")
    except Exception as e:
        local_conn.rollback()
        print(f"Error loading data into {table_name}: {e}")
    finally:
        cursor.close()

# Define table name and load data
table_name = 'train_schedule'
#load_data_to_db(df, azure_conn, table_name)
load_data_to_db(local_conn, table_name)

# Close connections
#azure_conn.close()
local_conn.close()


TypeError: string indices must be integers, not 'list'